# Double Debiased Machine Learning for Continuous Treatments

Prof. Daniel de Abreu Pereira Uhr

## Conteúdo

* Double Debiased Machine Learning for Continuous Treatments - DDML for Continuous Treatments


## Referências

**Principais:**
* Colangelo and Lee (2023). Double Debiased Machine Learning Nonparametric Inference with Continuous Treatments. [arXiv:2004.03036v8 ](https://doi.org/10.48550/arXiv.2004.03036) 
* GitHub: https://github.com/KColangelo/Double-ML-Continuous-Treatment
* 
**Complementares:**
* Microsoft EconML: https://econml.azurewebsites.net/
* UBER CausalML: https://causalml.readthedocs.io/en/latest/
* https://docs.doubleml.org/stable/index.html
* https://github.com/MasaAsami/ReproducingDMLDiD/blob/main/notebook/Reproduction_of_DMLDiD_RO_for_NEW_SIMDATA.ipynb


# Double Debiased Machine Learning for Continuous Treatments

## Introdução

O **Double Debiased Machine Learning (DML)** é um método estatístico moderno para realizar inferência causal em tratamentos contínuos. Ele combina técnicas de aprendizado de máquina e estatística não paramétrica, permitindo a estimativa eficiente de efeitos causais mesmo em cenários com covariáveis de alta dimensionalidade. 

O DML é robusto a especificações incorretas de funções auxiliares (nuisance functions), utilizando momentos duplamente robustos e "cross-fitting".

---

## Estrutura do Modelo

### Contexto
Considere uma amostra de $n$ observações $\{Y_i, T_i, X_i\}_{i=1}^n$, onde:
- $Y_i$: Desfecho de interesse.
- $T_i$: Tratamento contínuo.
- $X_i$: Conjunto de covariáveis observadas.

O objetivo é estimar a **Função de Resposta Média à Dose (Average Dose-Response Function - ADRF)**:
$$
\beta_t = \mathbb{E}[Y(t)],
$$
onde $Y(t)$ é o desfecho potencial para um valor específico $t$ do tratamento.

Além disso, podemos calcular o **efeito marginal parcial**:
$$
\theta_t = \frac{\partial \beta_t}{\partial t}.
$$

### Suposições
1. **Independência Condicional**:
   $$
   T \perp \varepsilon \mid X,
   $$
   onde $\varepsilon$ é o termo de erro não observado.

2. **Suporte Comum**:
   $$
   f_{T|X}(t \mid X) > 0 \quad \text{para todo } t \in \mathcal{T}.
   $$

---

## Método DML

O método DML baseia-se na construção de uma **função momento duplamente robusta** para estimar $\beta_t$. A função momento é definida como:
$$
\psi_t(Y_i, T_i, X_i) = \gamma(t, X_i) + \frac{K_h(T_i - t)}{f_{T|X}(t \mid X_i)} \left(Y_i - \gamma(t, X_i)\right),
$$
onde:
- $\gamma(t, X) = \mathbb{E}[Y \mid T = t, X]$: Expectativa condicional.
- $f_{T|X}(t \mid X)$: Densidade condicional do tratamento.
- $K_h(T_i - t)$: Função kernel para ponderar as observações próximas de $t$.

O estimador DML para $\beta_t$ é:
$$
\hat{\beta}_t = \frac{1}{n} \sum_{i=1}^n \psi_t(Y_i, T_i, X_i).
$$

### Propriedades do DML
1. **Double Robustness**:
   - O estimador permanece consistente se **apenas um** dos modelos auxiliares ($\gamma$ ou $f_{T|X}$) estiver corretamente especificado.

2. **Cross-Fitting**:
   - Divide os dados em $L$ folds para evitar overfitting, estimando $\gamma$ e $f_{T|X}$ em folds separados.

3. **Normalidade Assintótica**:
   - Sob condições regulares, $\hat{\beta}_t$ é assintoticamente normal:
     $$
     \sqrt{n} (\hat{\beta}_t - \beta_t) \xrightarrow{d} N(0, V_t),
     $$
     onde $V_t$ é a variância assintótica.

---

## Estimação do Efeito Marginal ($\theta_t$)

Para estimar o efeito marginal $\theta_t$, usamos derivação numérica:
$$
\hat{\theta}_t = \frac{\hat{\beta}_{t + \eta/2} - \hat{\beta}_{t - \eta/2}}{\eta},
$$
onde $\eta$ é uma sequência positiva que converge para zero à medida que $n \to \infty$.

---

## Benefícios e Limitações

### Benefícios
- **Flexibilidade**: Permite o uso de métodos de aprendizado de máquina para modelar $\gamma$ e $f_{T|X}$.
- **Eficiência**: Utiliza técnicas como cross-fitting para melhorar a precisão das estimativas.
- **Robustez**: Resistente a erros de especificação em uma das funções auxiliares.

### Limitações
- **Complexidade Computacional**: Requer a estimativa de funções auxiliares de alta dimensionalidade.
- **Sensibilidade ao Kernel**: A escolha do kernel e da largura de banda ($h$) pode impactar os resultados.

---

## Exemplo Intuitivo

Considere um estudo que analisa o impacto de **horas de treinamento em um programa de capacitação** ($T$) nos **salários futuros** ($Y$):
- Usamos $\gamma(t, X)$ para prever salários dados $T$ e as covariáveis $X$ (e.g., idade, escolaridade).
- Estimamos $f_{T|X}(t \mid X)$, que captura a distribuição das horas de treinamento, dado o perfil do indivíduo.
- O DML ajusta as estimativas para isolar o impacto causal de $T$ em $Y$, mesmo que algumas relações sejam complexas ou não lineares.

---

## Conclusão

O método DML é uma ferramenta poderosa para inferência causal em tratamentos contínuos. Ele combina robustez estatística com flexibilidade computacional, tornando-se ideal para aplicações em que as suposições tradicionais de modelos paramétricos são difíceis de satisfazer.

---
